In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
import joblib

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
fridge = pd.read_excel('emlak_gercek.xlsx')
df = fridge.copy()
df = df.dropna()
y = df["Fiyat"].astype('int32')
X_ = df.drop(['Fiyat', "Isıtma_Tipi", "Krediye_Uygunluk", "Eşya_Durumu", "Kira_Getirisi"], axis=1).astype('int32')
X = pd.DataFrame(X_)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                   random_state=42)

In [9]:
pip install catboost


  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/e2/63/379617e3d982e8a66c9d66ebf4621d3357c7c18ad356473c335bffd5aba6/catboost-1.2.2-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/47.0 kB ? eta -:--:--
     ---------------- --------------------- 20.5/47.0 kB 217.9 kB/s eta 0:00:01
     ------------------------ ------------- 30.7/47.0 kB 217.9 kB/s eta 0:00:01
     -------------------------------------- 47.0/47.0 kB 235.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/101.0 MB ? eta -:--:--
   ---------------------------------------- 0.2/101.0 MB 5.9 MB/s eta 0:00:17
   ---------------------------------------- 0.7/101.0 MB 7.4 MB/s eta 0:00:14
   ---------------------------------------- 1.1/101.0 MB 7.9 MB/s eta 0:00:13
    --------------------------------------- 1.9/101.0 MB 9.9 MB/s eta 0:00:11
   - -------------------------------------- 2.8/101.0 MB 10.4 MB/s eta 0:00:10
   - ----------

In [10]:
from catboost import CatBoostRegressor

In [11]:
catb = CatBoostRegressor()
catb_model = catb.fit(X_train, y_train)

Learning rate set to 0.047007
0:	learn: 8131096.4449443	total: 148ms	remaining: 2m 27s
1:	learn: 7769637.2610132	total: 150ms	remaining: 1m 14s
2:	learn: 7434411.5459349	total: 153ms	remaining: 50.9s
3:	learn: 7123470.5854009	total: 155ms	remaining: 38.7s
4:	learn: 6816586.2004261	total: 157ms	remaining: 31.3s
5:	learn: 6517413.5817261	total: 159ms	remaining: 26.4s
6:	learn: 6242289.3524660	total: 162ms	remaining: 22.9s
7:	learn: 5971714.2514009	total: 164ms	remaining: 20.3s
8:	learn: 5726460.8904079	total: 173ms	remaining: 19.1s
9:	learn: 5479990.1933674	total: 175ms	remaining: 17.4s
10:	learn: 5249672.3668945	total: 177ms	remaining: 15.9s
11:	learn: 5028059.5072630	total: 180ms	remaining: 14.9s
12:	learn: 4821004.3212721	total: 183ms	remaining: 13.9s
13:	learn: 4617827.6999818	total: 185ms	remaining: 13s
14:	learn: 4426185.8236983	total: 187ms	remaining: 12.3s
15:	learn: 4246858.2096553	total: 190ms	remaining: 11.7s
16:	learn: 4074779.0241305	total: 193ms	remaining: 11.2s
17:	learn: 

In [12]:
y_pred = catb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

1161450.7615882931

In [13]:
catb_grid = {
    'iterations': [200,500,1000,2000],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'depth': [3,4,5,6,7,8] }

In [14]:
catb = CatBoostRegressor()
catb_cv_model = GridSearchCV(catb, catb_grid, cv=5, n_jobs = -1, verbose = 2)

In [15]:
catb_cv_model.fit(X_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
0:	learn: 8410162.6906003	total: 3.71ms	remaining: 3.7s
1:	learn: 8331181.5405890	total: 7.14ms	remaining: 3.56s
2:	learn: 8253603.7890622	total: 12.5ms	remaining: 4.15s
3:	learn: 8176300.7220279	total: 17.2ms	remaining: 4.28s
4:	learn: 8101039.9327798	total: 21.6ms	remaining: 4.3s
5:	learn: 8026817.6911453	total: 22.6ms	remaining: 3.74s
6:	learn: 7955841.6757452	total: 26.8ms	remaining: 3.79s
7:	learn: 7882092.9570187	total: 29.1ms	remaining: 3.61s
8:	learn: 7808621.8902576	total: 32.2ms	remaining: 3.55s
9:	learn: 7734471.0104710	total: 35.1ms	remaining: 3.48s
10:	learn: 7662876.9958355	total: 36.8ms	remaining: 3.31s
11:	learn: 7591993.1438634	total: 42.5ms	remaining: 3.5s
12:	learn: 7521176.7275115	total: 46.1ms	remaining: 3.5s
13:	learn: 7450069.0519268	total: 47.3ms	remaining: 3.33s
14:	learn: 7380745.9696643	total: 50.2ms	remaining: 3.3s
15:	learn: 7314120.4948597	total: 51.3ms	remaining: 3.15s
16:	learn: 7246806.766074

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x0000024384137AD0>,
             n_jobs=-1,
             param_grid={'depth': [3, 4, 5, 6, 7, 8],
                         'iterations': [200, 500, 1000, 2000],
                         'learning_rate': [0.01, 0.03, 0.05, 0.1]},
             verbose=2)

In [16]:
catb_cv_model.best_params_

{'depth': 8, 'iterations': 1000, 'learning_rate': 0.01}

In [17]:
catb_tuned = CatBoostRegressor(iterations = 100, 
                               learning_rate = 0.01, 
                               depth = 8)

catb_tuned = catb_tuned.fit(X_train,y_train)

0:	learn: 8410162.6906003	total: 2.69ms	remaining: 266ms
1:	learn: 8331181.5405890	total: 5.47ms	remaining: 268ms
2:	learn: 8253603.7890622	total: 8.61ms	remaining: 279ms
3:	learn: 8176300.7220279	total: 12.3ms	remaining: 294ms
4:	learn: 8101039.9327798	total: 15.1ms	remaining: 287ms
5:	learn: 8026817.6911453	total: 16ms	remaining: 250ms
6:	learn: 7955841.6757452	total: 18.9ms	remaining: 251ms
7:	learn: 7882092.9570187	total: 20.2ms	remaining: 233ms
8:	learn: 7808621.8902576	total: 23.9ms	remaining: 241ms
9:	learn: 7734471.0104710	total: 26.6ms	remaining: 240ms
10:	learn: 7662876.9958355	total: 28.1ms	remaining: 227ms
11:	learn: 7591993.1438634	total: 30.9ms	remaining: 227ms
12:	learn: 7521176.7275115	total: 34.1ms	remaining: 228ms
13:	learn: 7450069.0519268	total: 35.4ms	remaining: 218ms
14:	learn: 7380745.9696643	total: 38.1ms	remaining: 216ms
15:	learn: 7314120.4948597	total: 38.9ms	remaining: 204ms
16:	learn: 7246806.7660747	total: 42.5ms	remaining: 207ms
17:	learn: 7181446.8981181

In [18]:
y_pred = catb_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

3325354.644175492

In [19]:
print("r^2 :", r2_score(y_test,y_pred))

r^2 : 0.8176268031505476
